In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
from sklearn.preprocessing import StandardScaler
'''
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_percentage_error
'''
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_percentage_error


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e1/sample_submission.csv
/kaggle/input/playground-series-s5e1/train.csv
/kaggle/input/playground-series-s5e1/test.csv


In [2]:
# Impoting sticker sales dataset
stickers_data = pd.read_csv("/kaggle/input/playground-series-s5e1/train.csv")

In [3]:
# Checking first five rows
stickers_data.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


## Knowing the dataset

In [4]:
# Checking shape
print(f"Number of rows: {stickers_data.shape[0]}")
print(f"Number of columns: {stickers_data.shape[1]}")

Number of rows: 230130
Number of columns: 6


In [5]:
print(f"Date range: {stickers_data['date'].min()} - {stickers_data['date'].max()}")

Date range: 2010-01-01 - 2016-12-31


7 years of data

In [6]:
# Columns in the dataset
stickers_data.columns

Index(['id', 'date', 'country', 'store', 'product', 'num_sold'], dtype='object')

In [7]:
# Checking null values
stickers_data.isnull().sum()

id             0
date           0
country        0
store          0
product        0
num_sold    8871
dtype: int64

In [8]:
# Statistical insight on dataset
stickers_data.describe()

,id,num_sold
count,230130.000000,221259.000000
mean,115064.500000,752.527382
std,66432.953062,690.165445
min,0.000000,5.000000
25%,57532.250000,219.000000
50%,115064.500000,605.000000
75%,172596.750000,1114.000000
max,230129.000000,5939.000000


In [9]:
# General insight on dataset
stickers_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null  object 
 4   product   230130 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.5+ MB


Inference till now:
1. Date column is in object dtype.
2. Null Values in "num_sold" (target) column.

Checking distribution of data per category

In [10]:
# Check unique values in categorical columns
print(stickers_data['country'].unique())
print(stickers_data['store'].unique())
print(stickers_data['product'].unique())

print()

# Check number of unique values in categorical columns
print(stickers_data['country'].nunique())
print(stickers_data['store'].nunique())
print(stickers_data['product'].nunique())

print()

# Check data distribution of unique values in categorical columns
print(stickers_data['country'].value_counts())
print()
print(stickers_data['store'].value_counts())
print()
print(stickers_data['product'].value_counts())

['Canada' 'Finland' 'Italy' 'Kenya' 'Norway' 'Singapore']
['Discount Stickers' 'Stickers for Less' 'Premium Sticker Mart']
['Holographic Goose' 'Kaggle' 'Kaggle Tiers' 'Kerneler'
 'Kerneler Dark Mode']

6
3
5

country
Canada       38355
Finland      38355
Italy        38355
Kenya        38355
Norway       38355
Singapore    38355
Name: count, dtype: int64

store
Discount Stickers       76710
Stickers for Less       76710
Premium Sticker Mart    76710
Name: count, dtype: int64

product
Holographic Goose     46026
Kaggle                46026
Kaggle Tiers          46026
Kerneler              46026
Kerneler Dark Mode    46026
Name: count, dtype: int64


Inference: Data seems fairly distributed per category.

### Need to do the following 
1. Drop "id" column. [Not of use as there are as mny unique values as many data points]
2. Null value treatment.
3. Change date column to datetime data type. [Curretly in Object datatype]
4. Extract date features like day, month, year and weekday.
5. Set the 'date' column as index after extracting the feature.
6. Encode 'country', 'store' and 'product' column.
7. Scale 'num_sold' (target) column.

### Data pre-processing

In [11]:
# 1. Dropping "id" column
stickers_data = stickers_data.drop("id", axis =1)
stickers_data.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,date,country,store,product,num_sold
0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [12]:
# 1. Null value treatment

# Creating a new column 'category' combining all the category
stickers_data['category'] = stickers_data['country'] + "_" + stickers_data['store'] + "_" + stickers_data['product'] 

# Value count by category
print("Value count by category")
print(stickers_data['category'].value_counts())

print()

# Null value count by category
print("Null Value count by category")
print(stickers_data[stickers_data['num_sold'].isna()]['category'].value_counts())

Value count by category
category
Canada_Discount Stickers_Holographic Goose           2557
Norway_Stickers for Less_Kaggle Tiers                2557
Norway_Stickers for Less_Holographic Goose           2557
Norway_Discount Stickers_Kerneler Dark Mode          2557
Norway_Discount Stickers_Kerneler                    2557
                                                     ... 
Finland_Premium Sticker Mart_Kerneler                2557
Finland_Premium Sticker Mart_Kaggle Tiers            2557
Finland_Premium Sticker Mart_Kaggle                  2557
Finland_Premium Sticker Mart_Holographic Goose       2557
Singapore_Premium Sticker Mart_Kerneler Dark Mode    2557
Name: count, Length: 90, dtype: int64

Null Value count by category
category
Canada_Discount Stickers_Holographic Goose       2557
Kenya_Discount Stickers_Holographic Goose        2557
Kenya_Stickers for Less_Holographic Goose        1358
Canada_Stickers for Less_Holographic Goose       1308
Kenya_Premium Sticker Mart_Holograph

Inference: Two of the categories have no values at all. Best to drop null values rather than imputing with forward/backward fill method or global mean.

In [13]:
# Dropping null values
stickers_data = stickers_data.dropna()

# Dropping 'category' column
stickers_data = stickers_data.drop('category', axis=1)

print(stickers_data.isna().sum())
print()
print(stickers_data.head())

date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

         date country              store             product  num_sold
1  2010-01-01  Canada  Discount Stickers              Kaggle     973.0
2  2010-01-01  Canada  Discount Stickers        Kaggle Tiers     906.0
3  2010-01-01  Canada  Discount Stickers            Kerneler     423.0
4  2010-01-01  Canada  Discount Stickers  Kerneler Dark Mode     491.0
5  2010-01-01  Canada  Stickers for Less   Holographic Goose     300.0


In [14]:
# 3. Converting "date" column to datetime datatype
stickers_data['date'] = pd.to_datetime(stickers_data['date'])
stickers_data['date'].dtype

dtype('<M8[ns]')

In [15]:
# 4. Extracting Date features like day, month, year and weekday
stickers_data['day'] = stickers_data['date'].dt.day
stickers_data['month'] = stickers_data['date'].dt.month
stickers_data['year'] = stickers_data['date'].dt.year
stickers_data['weekday'] = stickers_data['date'].dt.weekday

stickers_data.head()

,date,country,store,product,num_sold,day,month,year,weekday
1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0,1,1,2010,4
2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0,1,1,2010,4
3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0,1,1,2010,4
4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0,1,1,2010,4
5,2010-01-01,Canada,Stickers for Less,Holographic Goose,300.0,1,1,2010,4


In [16]:
# 5. Setting date column as index column.

stickers_data = stickers_data.set_index('date')

stickers_data.head()

,country,store,product,num_sold,day,month,year,weekday
date,,,,,,,,
2010-01-01,Canada,Discount Stickers,Kaggle,973.0,1,1,2010,4
2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0,1,1,2010,4
2010-01-01,Canada,Discount Stickers,Kerneler,423.0,1,1,2010,4
2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0,1,1,2010,4
2010-01-01,Canada,Stickers for Less,Holographic Goose,300.0,1,1,2010,4


In [17]:
# 6. Encode 'country', 'store' and 'product' column.

# Creating a function to encode columns.
def encode_column(dataset, column_name):
    var = pd.get_dummies(dataset[column_name])
    dataset = pd.concat([dataset, var], axis=1)
    dataset = dataset.drop(column_name, axis=1)
    return dataset

# Encoding columns
stickers_data = encode_column(stickers_data, 'country') # Country column
stickers_data = encode_column(stickers_data, 'store')   # Store column
stickers_data = encode_column(stickers_data, 'product') # Product column

stickers_data.head()

,num_sold,day,month,year,weekday,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less,Holographic Goose,Kaggle,Kaggle Tiers,Kerneler,Kerneler Dark Mode
date,,,,,,,,,,,,,,,,,,,
2010-01-01,973.0,1,1,2010,4,True,False,False,False,False,False,True,False,False,False,True,False,False,False
2010-01-01,906.0,1,1,2010,4,True,False,False,False,False,False,True,False,False,False,False,True,False,False
2010-01-01,423.0,1,1,2010,4,True,False,False,False,False,False,True,False,False,False,False,False,True,False
2010-01-01,491.0,1,1,2010,4,True,False,False,False,False,False,True,False,False,False,False,False,False,True
2010-01-01,300.0,1,1,2010,4,True,False,False,False,False,False,False,False,True,True,False,False,False,False


### Independent-Dependent || Train-Test Split

In [18]:
# Split data into training and testing sets
train = stickers_data['2010-01-01':'2015-12-31']   # Training with 6 years of data
test = stickers_data['2016-01-01':'2016-12-31']    # Testing with 1 year of data

In [19]:
'''
This one is for ANN model

# Separate features and target
X_train = np.array(train.drop('num_sold', axis=1), dtype=np.float32)
y_train = np.array(train['num_sold'], dtype=np.float32)
X_test = np.array(test.drop('num_sold', axis=1), dtype=np.float32)
y_test = np.array(test['num_sold'], dtype=np.float32)

print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'X_test shape: {y_test.shape}')
'''

"\nThis one is for ANN model\n\n# Separate features and target\nX_train = np.array(train.drop('num_sold', axis=1), dtype=np.float32)\ny_train = np.array(train['num_sold'], dtype=np.float32)\nX_test = np.array(test.drop('num_sold', axis=1), dtype=np.float32)\ny_test = np.array(test['num_sold'], dtype=np.float32)\n\nprint(f'X_train shape: {X_train.shape}')\nprint(f'y_train shape: {y_train.shape}')\nprint(f'X_test shape: {X_test.shape}')\nprint(f'X_test shape: {y_test.shape}')\n"

In [20]:
# For Random Forest model

# Separate features and target
X_train = train.drop('num_sold', axis=1)
y_train = train['num_sold']
X_test = test.drop('num_sold', axis=1)
y_test = test['num_sold']

print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'X_test shape: {y_test.shape}')

X_train shape: (189492, 18)
y_train shape: (189492,)
X_test shape: (31767, 18)
X_test shape: (31767,)


In [21]:
y_train

date
2010-01-01     973.0
2010-01-01     906.0
2010-01-01     423.0
2010-01-01     491.0
2010-01-01     300.0
               ...  
2015-12-31     387.0
2015-12-31    2224.0
2015-12-31    1995.0
2015-12-31    1110.0
2015-12-31    1175.0
Name: num_sold, Length: 189492, dtype: float64

In [22]:
# Scaling data

# Innitializing scaler
x_scaler = StandardScaler()
y_scaler = StandardScaler()

# Scaling X
x_scaler.fit(X_train)
X_train = x_scaler.transform(X_train)
X_test = x_scaler.transform(X_test)

# Scaling y
y_scaler.fit(y_train.values.reshape(-1, 1))
y_train = y_scaler.transform(y_train.values.reshape(-1, 1))
y_test = y_scaler.transform(y_test.values.reshape(-1, 1))

### Creating/Importing and training model

In [23]:
'''
# Input shape for Ann model
input_shape = X_train[0].shape

# Creating model
ann_model = Sequential([
    Dense(64, activation='relu', input_shape=(input_shape)),
    Dense(32, activation='relu'),
    Dense(1)    
])

# Configuring model
ann_model.compile(optimizer="adam", loss="mae", metrics=["mape"])
'''

'\n# Input shape for Ann model\ninput_shape = X_train[0].shape\n\n# Creating model\nann_model = Sequential([\n    Dense(64, activation=\'relu\', input_shape=(input_shape)),\n    Dense(32, activation=\'relu\'),\n    Dense(1)    \n])\n\n# Configuring model\nann_model.compile(optimizer="adam", loss="mae", metrics=["mape"])\n'

In [24]:
'''
# Prediction
y_pred = model.predict(X_test)

# Inverse scaling
y_pred = scaler.inverse_transform(y_pred)

y_pred = y_pred.flatten()
y_test = scaler.inverse_transform(y_test.reshape(-1,1)).flatten()
results = pd.DataFrame({'Predicted':y_pred, 'Actual':y_test})

from sklearn.metrics import r2_score
score = r2_score(y_pred, y_test)
print(f'ANN Model's score: {score})
print()
results
'''

"\n# Prediction\ny_pred = model.predict(X_test)\n\n# Inverse scaling\ny_pred = scaler.inverse_transform(y_pred)\n\ny_pred = y_pred.flatten()\ny_test = scaler.inverse_transform(y_test.reshape(-1,1)).flatten()\nresults = pd.DataFrame({'Predicted':y_pred, 'Actual':y_test})\n\nfrom sklearn.metrics import r2_score\nscore = r2_score(y_pred, y_test)\nprint(f'ANN Model's score: {score})\nprint()\nresults\n"

In [25]:
# Innitializing Random Forest Regressor
rf_model = RandomForestRegressor()

# Fitting data/Training model
rf_model.fit(X_train, y_train)

<ipython-input-25-45bb3e219586>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


RandomForestRegressor()

In [26]:
# Prediction
y_pred = rf_model.predict(X_test)

In [27]:
y_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))
y_pred
y_test = y_scaler.inverse_transform(y_test.reshape(-1, 1))
y_test

array([[ 706.],
       [ 634.],
       [ 316.],
       ...,
       [2299.],
       [1242.],
       [1622.]])

In [28]:
# Scoring

# R2 score
r2score = r2_score(y_pred, y_test)
print(f'Accuracy of the model is: {r2score}')

# Mean Absolute Percentage Error
mape = mean_absolute_percentage_error(y_pred, y_test)
print(f'Mean-Absolute-Percentage_error of the model is: {mape}')

Accuracy of the model is: 0.974317667264714
Mean-Absolute-Percentage_error of the model is: 0.07470096137158651


### Preiction on competitions test set

In [29]:
# Reading the Kaggle provided test set
validation_set = pd.read_csv("/kaggle/input/playground-series-s5e1/test.csv")
validation_set.head()

,id,date,country,store,product
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode


In [30]:
# Making a copy for preprocessing
val_set_copy = validation_set.copy()
val_set_copy

,id,date,country,store,product
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode
...,...,...,...,...,...
98545,328675,2019-12-31,Singapore,Premium Sticker Mart,Holographic Goose
98546,328676,2019-12-31,Singapore,Premium Sticker Mart,Kaggle
98547,328677,2019-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers
98548,328678,2019-12-31,Singapore,Premium Sticker Mart,Kerneler


In [31]:
# Creating a function for preprocessing
def preprocessing(df):
    df = df.drop('id', axis=1)
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['weekday'] = df['date'].dt.weekday
    df = df.drop('date', axis=1)
    df = encode_column(df, 'country') # Country column
    df = encode_column(df, 'store')   # Store column
    df = encode_column(df, 'product') # Product column
    df = x_scaler.transform(df)
    return df   

In [32]:
# Preprocessing the datast copy
val_set_copy = preprocessing(val_set_copy)

In [33]:
# Making predictions
val_pred = rf_model.predict(val_set_copy)

# Inverse transforming
val_pred = y_scaler.inverse_transform(val_pred.reshape(-1, 1))
val_pred

array([[ 147.79],
       [ 784.89],
       [ 611.47],
       ...,
       [2042.38],
       [1079.69],
       [1209.13]])

In [34]:
# Converting into series to concat into validation dataset
val_pred = pd.Series(val_pred.flatten())

In [35]:
# Creating a submission set
submission_set = pd.concat([validation_set, val_pred], axis=1)

In [36]:
# Dropping irrelevent columns
submission_set = submission_set.drop(['date', 'country', 'store', 'product'], axis=1)

In [37]:
# Finally how it looks
submission_set

,id,0
0,230130,147.79
1,230131,784.89
2,230132,611.47
3,230133,357.81
4,230134,421.11
...,...,...
98545,328675,366.13
98546,328676,2316.17
98547,328677,2042.38
98548,328678,1079.69


In [38]:
# Writing to a csv
submission_set.to_csv('submission.csv', index=False)